#### ¿Cual es el desempeño en las clases estudiantiles y la asignación correspondiente para un nuevo alumno?
Es fundamental para una institución reconocer los compartamientos académicos de sus estudiantes, por ello se requiere de un estudio que ayude a comparar las distintas clases y a la organización correspondiente de los alumnos en estas últimas, además, se requiere automatizar el proceso de asignar a cada alumno su correspondiente clase con el objetivo de reducir la inversión en tiempo y recursos y eliminar un potencial margen de error humano.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


df = pd.read_csv("data/student_performance.csv")

for c in df.columns[1:-1]:
    unique_values = df[c].unique()
    if len(unique_values) == 2:
        print(f"Valores únicos en '{c}': {df[c].unique()} (Binario)\n---------------------------------------------------------------------------------------------------------")
    else:
        print(f"Valores únicos en '{c}': {df[c].unique()}\n---------------------------------------------------------------------------------------------------------")

#### Dsitribución de frecuencia en categóricas binarias

In [ ]:
def percentage_binary(df, variable, category_1, category_2):
    variable_mean = df.groupby("GradeClass")[variable].mean().reset_index(name=f"{variable}_mean")
    variable_mean[f"{variable}_mean"] = variable_mean[f"{variable}_mean"] * 100 
    for v in variable_mean.values:
        percentage = 100 - v[1]
        if percentage >= 50:
            percentage = percentage.astype(str)
            print(f"En el grado clase {v[0]} el {percentage[:4]}% de los alumnos {category_1}")
        else:
            percentage = v[1].astype(str)
            print(f"En el grado clase {v[0]} el {percentage[:4]}% de los alumnos {category_2}")
    print("-------------------------------------------------------------------------------------------------------------")
            
gender_percentage = percentage_binary(df,"Gender","son Hombres","son Mujeres")
tutoring_percentage = percentage_binary(df,"Tutoring","no tiene tutoría","tiene tutoría")
extracurricular_percentage = percentage_binary(df,"Extracurricular","no participa en actividades extracurriculares","participa actividades extracurriculares")
sports_percentage = percentage_binary(df,"Sports","no participa en deportes","participa en deportes")
music_percentage = percentage_binary(df,"Music","no participa en actividades musicales","participa en actividades musicales")
volunteering_percentage = percentage_binary(df,"Volunteering","no participa en voluntariado","participa en voluntariado")

#### Distribución de frecuencia en variables categóricas (nominales y ordinales)

In [ ]:
def percentage(df, variable, categories):
    unique_values = df["GradeClass"].unique()
    for g in np.sort(unique_values):
        grade_class = df[df["GradeClass"] == g]
        variable_count = grade_class[variable].value_counts().reset_index()
        variable_count[variable] = categories
        print(f"{variable} en grado clase {g}")
        for p in variable_count.values:
            percentage = (p[1] / variable_count["count"].sum()) * 100
            percentage = percentage.astype(str)
            print(f"{p[0]}: {percentage[:4]}%")
        print("----------------------------------------")
        
ethnicity_percentage = percentage(df,"Ethnicity",["caucásicos","áfricanos","asiáticos","otros"])
parental_education_percentage = percentage(df,"ParentalEducation",["ninguno","escuela secundaria","un poco de universidad","licenciatura","superior"]) 
parental_support_percentage = percentage(df,"ParentalSupport",["ninguno","bajo","moderado","alto","muy alto"])

#### Naive Bayes 
Algoritmo de clasificación probabilístico basado en el teorema de Bayes y en la suposición de que las características son condicionalmente independientes entre si dado la clase a la que pertenecen, lo que significa que la probabilidad de una característica no depende de las otras, pero si aportan información sobre la variable objetivo. Además, supone que las variables independientes siguen una distribución específica, permitiendo calcular la probabilidad final de que la variable objetivo este asociada a esa clase. 

Suposiciones del algoritmo:
- Independencia condicional entre las características
- Distribuciones conocidas para modelar en las características(normal, binomial, etc.)
- No hay correlación entre características
- Clases mutuamente excluyentes
- No hay ruido en los datos

In [ ]:
# Discretización de la variable continua 'StudyTimeWeekly'
Q1 = np.percentile(df["StudyTimeWeekly"], 25)
Q2 = np.percentile(df["StudyTimeWeekly"], 50)
Q3 = np.percentile(df["StudyTimeWeekly"], 75)
Q4 = np.percentile(df["StudyTimeWeekly"], 100)

df["StudyTimeWeekly"] = np.where(df["StudyTimeWeekly"] <= Q1, Q1, 
                        np.where((df["StudyTimeWeekly"] > Q1) & (df["StudyTimeWeekly"] <= Q2), Q2, 
                        np.where((df["StudyTimeWeekly"] > Q2) & (df["StudyTimeWeekly"] <= Q3), Q3, 
                        df["StudyTimeWeekly"])))

df["StudyTimeWeekly"] = round(df["StudyTimeWeekly"])

#------------------------------------------------------------------------------------------------------

x_train, x_test, y_train, y_test = train_test_split(
                                                   df[df.columns[1:-1]],
                                                   df["GradeClass"],
                                                   test_size=0.2)

naive_bayes = GaussianNB()
naive_bayes.fit(x_train, y_train) 

class_predicts = naive_bayes.predict(x_test)
class_real = y_test.values

accuracy = accuracy_score(class_real, class_predicts)

x_test["GradeClass"] = class_real
class_real_count = x_test["GradeClass"].value_counts().reset_index()

class_predicts = pd.DataFrame(class_predicts, columns=["GradeClass"])
class_predicts_count = class_predicts["GradeClass"].value_counts().reset_index()

fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].pie(class_real_count["count"], labels=class_real_count["GradeClass"], autopct='%1.1f%%')
ax[0].set_title("Clases reales")
ax[1].pie(class_predicts_count["count"], labels=class_predicts_count["GradeClass"], autopct='%1.1f%%')
ax[1].set_title(f"Acertividad del modelo: {round(accuracy,2)}")

plt.subplots_adjust(wspace=0.2, hspace=0.3)
plt.show()

#### Dashboard que refleja las proporciones de datos académicos de los estudiantes en las distintas clases


In [ ]:
app = dash.Dash(__name__)

options_dropdown = [
    {"label":"Género","value":"Gender"},
    {"label":"Tutoría","value":"Tutoring"},
    {"label":"Actividades extracurriculares","value":"Extracurricular"},
    {"label":"Deportes","value":"Sports"},
    {"label":"Música","value":"Music"},
    {"label":"Voluntariado","value":"Volunteering"},
    {"label":"Etnia","value":"Ethnicity"},
    {"label":"Educación paternal","value":"ParentalEducation"},
    {"label":"Apoyo paternal","value":"ParentalSupport"}
]

app.layout = html.Div(id="body",className="e6_body",children=[
    html.H1("Categorías de estudiantes",id="title",className="e6_title"),
    html.Div(id="div",className="e6_div",children=[
        html.Div(id="div_dropdown",className="e6_div_dropdown",children=[
            dcc.Dropdown(id="dropdown_1",className="e6_dropdown",
                        options = [
                            {"label":"Clase de grado 0","value":0},
                            {"label":"Clase de grado 1","value":1},
                            {"label":"Clase de grado 2","value":2},
                            {"label":"Clase de grado 3","value":3},
                            {"label":"Clase de grado 4","value":4}
                        ],
                        value=0,
                        multi=False,
                        clearable=False),
            dcc.Dropdown(id="dropdown_2",className="e6_dropdown",
                        options=options_dropdown,
                        value="Gender",
                        multi=False,
                        clearable=False)
        ])
    ]),
    dcc.Graph(id="graph",className="e6_graph",figure={})
])

@app.callback(
    Output(component_id="graph",component_property="figure"),
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

def update_graph(slct_var_obj,slct_var_car):
    
    grade_class = df[df["GradeClass"] == slct_var_obj]
    variable_count = grade_class[slct_var_car].value_counts().reset_index()
    variable_count.sort_values(slct_var_car, inplace=True)
        
    if slct_var_car == "Gender":
        variable_count[slct_var_car] = ["Hombre","Mujer"]
    elif slct_var_car == "Tutoring":
        variable_count[slct_var_car] = ["Sin tutoría","Tutoría"]
    elif slct_var_car == "Extracurricular":
        variable_count[slct_var_car] = ["No participa en actividades extracurriculares","Participa en actividades extracurriculares"]
    elif slct_var_car == "Sports":
        variable_count[slct_var_car] = ["No participa en deportes","Participa en deportes"]
    elif slct_var_car == "Music":
        variable_count[slct_var_car] = ["No participa en actividades musicales","Participa en actividades musicales"]
    elif slct_var_car == "Volunteering":
        variable_count[slct_var_car] = ["No participa en voluntariado","Participa en voluntariado"]
    elif slct_var_car == "Ethnicity":
        variable_count[slct_var_car] = ["caucásicos","áfricanos","asiáticos","otros"]
    elif slct_var_car == "ParentalEducation":
        variable_count[slct_var_car] = ["ninguno","escuela secundaria","un poco de universidad","licenciatura","superior"]
    elif slct_var_car == "ParentalSupport":
        variable_count[slct_var_car] = ["ninguno","bajo","moderado","alto","muy alto"]
    
    selected_label = next(
        (option["label"] for option in options_dropdown if option["value"] == slct_var_car),
        "No encontrado" 
     )

    piechart = px.pie(variable_count, values="count", names=slct_var_car, title=f"Distribución de {selected_label} en clase de grado {slct_var_obj}")

    return piechart

if __name__ == "__main__":
    app.run_server(debug=False)